In [27]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import sys
import os
sys.path.insert(0, os.path.abspath(".."))
import pysgmcmc as pg

## 1. Instantiating a Sampler

To instantiate a sampler, we need two ingredients:
* Target parameters of the sampler: a list of `tensorflow.Variable` objects 
* A cost function: callable that maps these target parameters to a 1-d `tensorflow.Tensor` representing their corresponding costs

Note: In MCMC literature, the target parameters are often denoted as $\theta$ and the cost function is frequently referred to as $U(\theta)$.

In [28]:
# target parameters
parameters = [tf.Variable(0.), tf.Variable(0.)]

# cost function
# XXX: Use some meaningful cost function here
def cost_fun(params):
    raise NotImplementedError("...")
    

Given these ingredients, we can instantiate any of our samplers within a `tensorflow.Session`:

In [29]:
from pysgmcmc.samplers.sghmc import SGHMCSampler

session = tf.Session()

sampler = SGHMCSampler(
    params=parameters, cost_fun=cost_fun, session=session, dtype=tf.float32
)


NotImplementedError: ...

### Using data minibatches

TODO: Explain data_batches.py and how to use it

### Available samplers

TODO: INSERT HYPERLINK TO DOKU BELOW


To get an overview of which samplers are available for use, examine our [documentation](http://pysgmcmc.readthedocs.io/en/latest/) or simply run:


In [30]:
help(pg.samplers)

Help on package pysgmcmc.samplers in pysgmcmc:

NAME
    pysgmcmc.samplers

PACKAGE CONTENTS
    relativistic_hmc
    relativistic_hmc2
    relativistic_sghmc
    sghmc
    sgld
    svgd

CLASSES
    pysgmcmc.sampling.BurnInMCMCSampler(pysgmcmc.sampling.MCMCSampler)
        pysgmcmc.samplers.sghmc.SGHMCSampler
        pysgmcmc.samplers.sgld.SGLDSampler
    
    class SGHMCSampler(pysgmcmc.sampling.BurnInMCMCSampler)
     |  Stochastic Gradient Hamiltonian Monte-Carlo Sampler that uses a burn-in
     |  procedure to adapt its own hyperparameters during the initial stages
     |  of sampling.
     |  
     |  See [1] for more details on this burn-in procedure.
     |  See [2] for more details on Stochastic Gradient Hamiltonian Monte-Carlo.
     |  
     |  [1] J. T. Springenberg, A. Klein, S. Falkner, F. Hutter
     |      Bayesian Optimization with Robust Bayesian Neural Networks.
     |      In Advances in Neural Information Processing Systems 29 (2016).
     |  
     |  [2] T. Chen, E

#### Sampler hyperparameters

To get a clearer picture of all possible design choices when instantiating any of 
our samplers, consider our docstrings:

In [31]:
help(pg.samplers.SGHMCSampler)

Help on class SGHMCSampler in module pysgmcmc.samplers.sghmc:

class SGHMCSampler(pysgmcmc.sampling.BurnInMCMCSampler)
 |  Stochastic Gradient Hamiltonian Monte-Carlo Sampler that uses a burn-in
 |  procedure to adapt its own hyperparameters during the initial stages
 |  of sampling.
 |  
 |  See [1] for more details on this burn-in procedure.
 |  See [2] for more details on Stochastic Gradient Hamiltonian Monte-Carlo.
 |  
 |  [1] J. T. Springenberg, A. Klein, S. Falkner, F. Hutter
 |      Bayesian Optimization with Robust Bayesian Neural Networks.
 |      In Advances in Neural Information Processing Systems 29 (2016).
 |  
 |  [2] T. Chen, E. B. Fox, C. Guestrin
 |      Stochastic Gradient Hamiltonian Monte Carlo
 |      In Proceedings of Machine Learning Research 32 (2014).
 |  
 |  Method resolution order:
 |      SGHMCSampler
 |      pysgmcmc.sampling.BurnInMCMCSampler
 |      pysgmcmc.sampling.MCMCSampler
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(se

## 2. Extracting samples

Extracting the next sample (with corresponding costs) from any of our samplers always simply amounts to:

In [32]:
sample, cost = next(sampler)

NameError: name 'sampler' is not defined

This interface allows us to extract samples in different contexts:

1. extract a chain of n subsequent samples
2. sample until an external event occurs / an external condition becomes `true`

In [1]:
# 1. extract a chain of n subsequent samples
samples, n = [], 1000


for _ in range(n):
    sample, _ = next(sampler)
    samples.append(sample)

# shorthand for 1., using itertools.islice
import itertools
samples = [sample for sample, _ in itertools.islice(sampler, n)]
    
# 2. sample until an external event occurs

# dummy event
def external_event():
    return np.random.randint(0, 10) > 5

samples = []
while not external_event():
    sample, _ = next(sampler)
    samples.append(sample)
    
    
samples

NameError: name 'sampler' is not defined

This interface also allows us to use any of our samplers in (infinite) for-loops. 

But *be warned*: such a for-loop will **not terminate** unless you explicitly break out of it:

In [34]:
samples, i = [], 0
for sample, cost in sampler:
    if i > 10:
        break
    i += 1
    samples.append(sample)

NameError: name 'sampler' is not defined

## 3. Analyzing chains/traces of samples

To analyze the results of a sampler run, we transform the results obtained by our samplers into `pymc3.MultiTrace` objects. Then we can use the (well-established) `pymc3` machinery to compute diagnostics for our samples:

In [35]:
from pysgmcmc.diagnostics.sample_chains import PYSGMCMCTrace

# XXX: Compute PYSGMCMCTrace (and possibly pymc3.MultiTrace from those) and 
# use those to compute e.g. ess and maybe produce some plots too


For convenience we also provide a shortcut function that directly computes a multitrace for one of our samplers:

In [36]:
help(pg.diagnostics.sample_chains.pymc3_multitrace)

Help on function pymc3_multitrace in module pysgmcmc.diagnostics.sample_chains:

pymc3_multitrace(get_sampler, n_chains=2, samples_per_chain=100, parameter_names=None)
    Extract chains from `sampler` and return them as `pymc3.MultiTrace`
        object.
    
    Parameters
    ----------
    sampler : pysgmcmc.sampling.MCMCSampler subclass
        An instance of one of our samplers.
    
    parameter_names : List[String] or NoneType, optional
        List of names for each target parameter of the sampler.
        If set to `None`, simply enumerate the parameters and use those numbers
        as names.
        Defaults to `None`.
    
    Returns
    ----------
    multitrace : pymc3.backends.base.MultiTrace
        TODO: DOKU
    
    
    Examples
    ----------
    TODO ADD EXAMPLE



## 4. PYSGMCMC - trained BNN

We provide an implementation of a Bayesian Neural Network that is trained using our samplers. 

The (tensorflow-) architecture of this BNN can be customized by the user and any of our sampling methods can be used to sample networks during training.




In [37]:
# XXX: Showcase our BNN on some hpolib function here